# Script Adapatation vs Non SVM classification (default parameters, k = linear, C = 1) 
## Sliding window, concatonate windows, seperate conditions

In [1]:
from sklearnex import patch_sklearn

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import numpy as np
import scipy.io
import os

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.covariance import LedoitWolf

from pyriemann.estimation import Covariances
from pyriemann.classification import TSclassifier
from pyriemann.spatialfilters import CSP
from pyriemann.tangentspace import TangentSpace
#from pyriemann.classification import SVC

from sklearn.svm import SVC

import h5py

import time

In [3]:
start_time = time.time()

## Load datasets

In [4]:
#LoadingPath='.../Combined_Subjects/TrainingTestingDatasets/SeperateConditions/Delay/ZScored'
#LoadingPath='.../Combined_Subjects/TrainingTestingDatasets/SeperateConditions/EMG/RLR/Delay'
LoadingPath = '.../Combined_Subjects/Ns_100/Delay'

In [5]:
n_components = 108 #EEG
#n_components = 4 #EMG

### Load : TrainingDataset & TestingDataset

In [6]:
TrainingPath = 'TrainingDataset.mat'
TestingPath = 'TestingDataset.mat'

training_path = os.path.join(LoadingPath, TrainingPath)
testing_path = os.path.join(LoadingPath, TestingPath)

#### TrainingDataset

In [7]:
f_TrainingDataset = scipy.io.loadmat(training_path)
TrainingDataset = f_TrainingDataset['TrainingDataset']

X_train_dataset = TrainingDataset['x']
Y_train_dataset = TrainingDataset['y']

nested_array_X = X_train_dataset[0][0]
X_train_dataset = np.array(nested_array_X)
X_train_dataset= np.transpose(X_train_dataset, (2, 1, 0))

nested_array_Y = Y_train_dataset[0][0]
Y_train_dataset = np.array(nested_array_Y)
Y_train_dataset=Y_train_dataset.flatten()

In [8]:
print(X_train_dataset.shape)
print(Y_train_dataset.shape)

(48435, 108, 100)
(48435,)


#### TestingDataset

In [9]:
f_TestingDataset = scipy.io.loadmat(testing_path)
TestingDataset = f_TestingDataset['TestingDataset']

X_test_dataset = TestingDataset['x']
Y_test_dataset = TestingDataset['y']

nested_array_X = X_test_dataset[0][0]
X_test_dataset = np.array(nested_array_X)
X_test_dataset= np.transpose(X_test_dataset, (2, 1, 0))

nested_array_Y = Y_test_dataset[0][0]
Y_test_dataset = np.array(nested_array_Y)
Y_test_dataset=Y_test_dataset.flatten()

In [10]:
print(X_test_dataset.shape)
print(Y_test_dataset.shape)

(5381, 108, 100)
(5381,)


### Load : TrainingProjected & TestingProjected

In [11]:
TrainingProjectedPath = 'TrainingProjected.mat'
TestingProjectedPath = 'TestingProjected.mat'

trainingProjected_path = os.path.join(LoadingPath, TrainingProjectedPath)
testingProjected_path = os.path.join(LoadingPath, TestingProjectedPath)

#### TrainingProjected

In [12]:
f_TrainingProjected=scipy.io.loadmat(trainingProjected_path)

TrainingProjected = f_TrainingProjected['TrainingProjected']

X_train_projected = TrainingProjected['x']
Y_train_projected = TrainingProjected['y']

nested_array_X = X_train_projected[0][0]
X_train_projected = np.array(nested_array_X)
#X_train_projected = np.transpose(X_train_projected, (2, 1, 0))
X_train_projected = np.transpose(X_train_projected, (2, 0, 1))

#nested_array_Y = Y_train_projected[0][0]
nested_array_Y = Y_train_projected[0][0][0]
Y_train_dataset = np.array(nested_array_Y)
#Y_train_projected=Y_train_projected.flatten()
Y_train_projected=nested_array_Y

In [13]:
print(X_train_projected.shape)
print(Y_train_projected.shape)

(48435, 108, 100)
(48435,)


#### TestingProjected

In [14]:
f_TestingProjected = scipy.io.loadmat(testingProjected_path)
TestingProjected = f_TestingProjected['TestingProjected']

X_test_projected = TestingProjected['x']
Y_test_projected = TestingProjected['y']

nested_array_X = X_test_projected[0][0]
X_test_projected = np.array(nested_array_X)
X_test_projected= np.transpose(X_test_projected, (2, 0, 1))

nested_array_Y = Y_test_projected[0][0]
Y_test_projected = np.array(nested_array_Y)
Y_test_projected=Y_test_projected.flatten()

In [15]:
print(X_test_projected.shape)
print(Y_test_projected.shape)

(5381, 108, 100)
(5381,)


### Load : TrainingFeatures & TestingFeatures

In [16]:
TrainingFeaturesPath = 'TrainingFeatures.mat'
TestingFeaturesPath = 'TestingFeatures.mat'

trainingFeatures_path = os.path.join(LoadingPath, TrainingFeaturesPath)
testingFeatures_path = os.path.join(LoadingPath, TestingFeaturesPath)

#### TrainingFeatures

In [17]:
f_TrainingFeatures = scipy.io.loadmat(trainingFeatures_path)

TrainingFeatures = f_TrainingFeatures['TrainingFeatures']
#X_train_features = TrainingFeatures[:, :108]
X_train_features = TrainingFeatures[:, :n_components]

#Y_train_features = TrainingFeatures[:, 108:]
Y_train_features = TrainingFeatures[:, n_components:]
Y_train_features = Y_train_features.flatten()

In [18]:
print(X_train_features.shape)
print(Y_train_features.shape)

(48435, 108)
(48435,)


#### TestingFeatures

In [19]:
f_TestingFeatures = scipy.io.loadmat(testingFeatures_path)

TestingFeatures = f_TestingFeatures['TestingFeatures']
#X_test_features = TestingFeatures[:, :108]
X_test_features = TestingFeatures[:, :n_components]

#Y_test_features = TestingFeatures[:, 108:]
Y_test_features = TestingFeatures[:, n_components:]
Y_test_features=Y_test_features.flatten()

In [20]:
print(X_test_features.shape)
print(Y_test_features.shape)

(5381, 108)
(5381,)


### Load: TrainingProjectedR & TestingProjectedR 
#### (If seperately found CSP & RCSP)

# Classification Algorithms

## "Riemannian geometry" (W/out CSP) 

In [21]:
cov_estimator = Covariances(estimator='lwf') # helps with regularisation 

X_train_cov=cov_estimator.transform(X_train_dataset)
# Apply learned transformation to testing dataset 
X_test_cov = cov_estimator.transform(X_test_dataset)

In [22]:
print(X_train_cov.shape)
print(X_test_cov.shape)

(48435, 108, 108)
(5381, 108, 108)


Using TSclassifier

In [23]:
clf_TS = TSclassifier(metric='riemann', clf=SVC(C=1.0, kernel='linear'))
clf_TS.fit(X_train_cov, Y_train_dataset)

# Make predictions on the test set
y_pred = clf_TS.predict(X_test_cov)

In [24]:
#Accuracy
accuracy = accuracy_score(Y_test_dataset, y_pred)
print(f"Accuracy: {accuracy * 100:.5f}%")

# Recall
recall = recall_score(Y_test_dataset, y_pred)  
print(f"Recall: {recall * 100:.5f}%")

# Precision
precision = precision_score(Y_test_dataset, y_pred)  
print(f"Precision: {precision * 100:.5f}%")

# F1 Score
f1 = f1_score(Y_test_dataset, y_pred)  
print(f"F1 Score: {f1 * 100:.5f}%")

Accuracy: 70.11708%
Recall: 68.77114%
Precision: 70.19563%
F1 Score: 69.47608%


## CSP: Arithmetic average "Spatial filtered (A) + Riemann"
### Project spatially filtered signals (ProjectedDataset) to tangent space 

In [25]:
# Find covariance matrix of X_train_projected
cov_estimator_projected = Covariances(estimator='lwf') # helps with regularisation 

X_train_cov_projected=cov_estimator_projected.transform(X_train_projected)
# Apply learned transformation to testing dataset 
X_test_cov_projected = cov_estimator_projected.transform(X_test_projected)

In [26]:
print(X_train_cov_projected.shape)
print(X_test_cov_projected.shape)

(48435, 108, 108)
(5381, 108, 108)


In [27]:
clf_TS_proj = TSclassifier(metric='riemann', clf=SVC(C=1.0, kernel='linear'))
clf_TS_proj.fit(X_train_cov_projected, Y_train_projected)

# Make predictions on the test set
y_pred_proj = clf_TS_proj.predict(X_test_cov_projected)

In [28]:
#Accuracy
accuracy = accuracy_score(Y_test_dataset, y_pred_proj)
print(f"Accuracy: {accuracy * 100:.5f}%")

# Recall
recall = recall_score(Y_test_dataset, y_pred_proj)  
print(f"Recall: {recall * 100:.5f}%")

# Precision
precision = precision_score(Y_test_dataset, y_pred_proj)  
print(f"Precision: {precision * 100:.5f}%")

# F1 Score
f1 = f1_score(Y_test_dataset, y_pred_proj)  
print(f"F1 Score: {f1 * 100:.5f}%")

Accuracy: 70.07991%
Recall: 70.46223%
Precision: 69.47017%
F1 Score: 69.96269%


## CSP: Arithmetic average "Spatial filtered (RCSP) (A) + Riemann"
### Project spatially filtered signals (ProjectedDataset_R) to tangent space 

## CSP: Riemannian average (1) "CSP(R) + Riemann"
### Use pyriemann.spatialfilters.csp (TrainingDataset) to obtain spatially filtered signals, project and classify in tangent space

In [29]:
csp = CSP(nfilter=n_components, metric="riemann", log=False)
cov_estimator = Covariances(estimator='lwf')

X_train_cov=cov_estimator.transform(X_train_dataset)
X_test_cov = cov_estimator.transform(X_test_dataset)

#csp.fit_transform takes covariance matrix as input 
X_train_csp = csp.fit_transform(X_train_cov, Y_train_dataset)

# Apply learned transformation to testing dataset 
X_test_csp = csp.transform(X_test_cov)

In [30]:
print(X_train_csp.shape)

(48435, 108, 108)


In [31]:
clf_TS_features = TSclassifier(metric='riemann', clf=SVC(C=1.0, kernel='linear'))
clf_TS_features.fit(X_train_csp, Y_train_dataset)

# Make predictions on the test set
y_pred_features = clf_TS_features.predict(X_test_csp)

In [32]:
#Accuracy
accuracy = accuracy_score(Y_test_dataset, y_pred_features)
print(f"Accuracy: {accuracy * 100:.5f}%")

# Recall
recall = recall_score(Y_test_dataset, y_pred_features)  
print(f"Recall: {recall * 100:.5f}%")

# Precision
precision = precision_score(Y_test_dataset, y_pred_features)  
print(f"Precision: {precision * 100:.5f}%")

# F1 Score
f1 = f1_score(Y_test_dataset, y_pred_features)  
print(f"F1 Score: {f1 * 100:.5f}%")

Accuracy: 70.11708%
Recall: 68.77114%
Precision: 70.19563%
F1 Score: 69.47608%


## CSP: Riemannian average (2) "CSP(R)"
### Classification w/out projection in TS

In [33]:
csp2 = CSP(nfilter=n_components, metric="riemann", log=True)
cov_estimator = Covariances(estimator='lwf')

X_train_cov=cov_estimator.transform(X_train_dataset)
X_test_cov = cov_estimator.transform(X_test_dataset)

#csp.fit_transform takes covariance matrix as input 
X_train_csp = csp2.fit_transform(X_train_cov, Y_train_dataset)

# Apply learned transformation to testing dataset 
X_test_csp = csp2.transform(X_test_cov)

In [34]:
print(X_train_csp.shape)

(48435, 108)


In [35]:
clf_csp = SVC(C=1.0, kernel='linear')
clf_csp.fit(X_train_csp, Y_train_dataset)

# Make predictions on the test set
y_pred_csp2 = clf_csp.predict(X_test_csp)

In [36]:
#Accuracy
accuracy = accuracy_score(Y_test_dataset, y_pred_csp2)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Recall
recall = recall_score(Y_test_dataset, y_pred_csp2)  
print(f"Recall: {recall * 100:.2f}%")

# Precision
precision = precision_score(Y_test_dataset, y_pred_csp2)  
print(f"Precision: {precision * 100:.2f}%")

# F1 Score
f1 = f1_score(Y_test_dataset, y_pred_csp2)  
print(f"F1 Score: {f1 * 100:.2f}%")

Accuracy: 59.69%
Recall: 57.84%
Precision: 59.51%
F1 Score: 58.66%


## CSP: Concatenation of features 
### Concatenate CSP variance features and tangent space features 

##### Find log-variance of spatially filtered matrices 

In [37]:
# loaded as X_train_features 
print(X_train_features.shape)

(48435, 108)


##### Find Tangent features 

In [38]:
cov_estimator = Covariances(estimator='lwf') # helps with regularisation 

X_train_cov=cov_estimator.transform(X_train_dataset)
# Apply learned transformation to testing dataset 
X_test_cov = cov_estimator.transform(X_test_dataset)

In [39]:
TSpace = TangentSpace(metric='riemann', tsupdate=False).fit(X_train_cov)
X_train_TS = TSpace.transform(X_train_cov)

X_test_TS = TSpace.transform(X_test_cov)

In [40]:
print(X_train_TS.shape) 

(48435, 5886)


##### Concatenate datasets 

In [41]:
X_train_cat=np.concatenate((X_train_features, X_train_TS), axis=1)

X_test_cat=np.concatenate((X_test_features, X_test_TS), axis=1)

##### Classify

In [42]:
clf_TS_cat = SVC(C=1.0, kernel='linear')
clf_TS_cat.fit(X_train_cat, Y_train_dataset)

# Make predictions on the test set
y_pred_cat = clf_TS_cat.predict(X_test_cat)

In [43]:
#Accuracy
accuracy = accuracy_score(Y_test_dataset, y_pred_cat)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Recall
recall = recall_score(Y_test_dataset, y_pred_cat)  
print(f"Recall: {recall * 100:.2f}%")

# Precision
precision = precision_score(Y_test_dataset, y_pred_cat)  
print(f"Precision: {precision * 100:.2f}%")

# F1 Score
f1 = f1_score(Y_test_dataset, y_pred_cat)  
print(f"F1 Score: {f1 * 100:.2f}%")

Accuracy: 70.90%
Recall: 71.10%
Precision: 70.36%
F1 Score: 70.73%


In [44]:
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time/60, "minutes")

Elapsed time: 691.8912134488423 minutes
